In [24]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import math
import requests
import numpy as np

In [15]:
options = webdriver.ChromeOptions()
# options.add_argument("--window-size=1920,1080")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options) 
driver.implicitly_wait(0.5)
link = "https://finance.discuss.com.hk/forumdisplay.php?fid=57&filter=type&orderby=new_lastpost&ascdesc=DESC&typeid=1293"
driver.get(link)
time.sleep(2.5)

data = {"Date":[], "Comments":[]}

for p in range(1, 5):
    if p > 0:
        link1 = f"{link}&page={p}"
        driver.get(link1)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        all_post = soup.find_all(lambda tag: tag.name == 'tbody' and tag.get('class') == ['forumdisplay_thread'])
        # all_post = soup.find_all("tbody", attrs={"class":"forumdisplay_thread"})

        for h in all_post:
            url = f"https://finance.discuss.com.hk/{h.span.a.get('href')}"
            driver.get(url)

            ads_elems = driver.find_elements(By.CSS_SELECTOR, ".adsbygoogle.adsbygoogle-noablate[data-vignette-loaded=true]")
            if len(ads_elems) > 0:
                ActionChains(driver).move_by_offset(5, 5).click().perform()
                time.sleep(5)

            # login_elems = driver.find_elements(By.CSS_SELECTOR, ".maintable.wrap.discuss_regist_wapper login.discuss_regist_section")
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            login_elems = soup.find("div", attrs={"class":"discuss_regist_section"})
            if login_elems != None:
                pass
                # driver.find_elements(By.CSS_SELECTOR, ".discuss_regist_section form input[name='username']")[0].send_keys(ID)
                # driver.find_elements(By.CSS_SELECTOR, ".discuss_regist_section form input[name='password']")[0].send_keys(password)
                # driver.find_elements(By.CSS_SELECTOR, ".discuss_regist_section form button[name='loginsubmit']")[0].click()
            else:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                head_comment = soup.find("meta", {"property":"og:description"})["content"]
                head_date = soup.find("div", attrs={"class":"post-date"}).text
                data["Date"].append(head_date)
                data["Comments"].append(head_comment)
                all_comments = soup.find("div", attrs={"class":"mainbox-container mb-t_msgfont"}).find_all("div", attrs={"class":"mainbox viewthread"})

                for c in all_comments:
                    if c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span") == None:
                        pass
                    else:
                        if c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span").find("div", attrs={"class":"quote"}) != None:
                            normal_comments = c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span")
                            unwanted = normal_comments.find("div", attrs={"class":"quote"})
                            unwanted.extract()
                            comment_date = c.find("div", attrs={"class":"post-date"}).text
                            data["Date"].append(comment_date)
                            data["Comments"].append(normal_comments.text)
                        elif c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span").find("div", attrs={"class":"quote"}) == None:
                            normal_comments = c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span").text
                            comment_date = c.find("div", attrs={"class":"post-date"}).text
                            data["Date"].append(comment_date)
                            data["Comments"].append(normal_comments)

                exist_pages = soup.find("div", attrs={"class":"pagination-buttons"})
                
                driver.switch_to.default_content()

                if exist_pages == None:
                    pass
                else:
                    no_pages = math.ceil((int(soup.find("div", attrs={"class":"pagination-buttons"}).find("em").text))/15)
                    for i in range(no_pages+1):
                        if i > 1:
                            # ads_elems = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.adsbygoogle.adsbygoogle-noablate[data-vignette-loaded=true]')))
                            ads_elems = driver.find_elements(By.CSS_SELECTOR, ".adsbygoogle.adsbygoogle-noablate[data-vignette-loaded=true]")
                            if len(ads_elems) > 0:
                                ActionChains(driver).move_by_offset(5, 5).click().perform()
                                time.sleep(5)
                            url2 = f'{url}&page={i}'
                            driver.get(url2)
                            soup = BeautifulSoup(driver.page_source, 'html.parser')
                            all_comments = soup.find_all("div", attrs={"class":"postmessage-content t_msgfont"})
                            comment_date = soup.find_all("div", attrs={"class":"post-date"})
                            for c in all_comments:
                                if c.find("span") == None:
                                    data["Comments"].append("NA")
                                elif c.find("span").find("div", attrs={"class":"quote"}) != None:
                                    comment = c.find("span")
                                    unwanted = comment.find("div", attrs={"class":"quote"})
                                    unwanted.extract()
                                    data["Comments"].append(comment.text)
                                elif c.find("span").find("div", attrs={"class": "quote"}) == None:
                                    comment = c.find("span").text
                                    data["Comments"].append(comment)
                            for d in comment_date:
                                data["Date"].append(d.text)
                                
                            driver.switch_to.default_content()



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [/Users/kayuenko/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache
/var/folders/ms/wg96lht949l8mqzyvc10vtv80000gn/T/ipykernel_45530/2673790649.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


WebDriverException: Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.53)
Stacktrace:
0   chromedriver                        0x0000000104651d14 chromedriver + 3792148
1   chromedriver                        0x00000001045e8828 chromedriver + 3360808
2   chromedriver                        0x00000001042e90b8 chromedriver + 217272
3   chromedriver                        0x00000001042d8124 chromedriver + 147748
4   chromedriver                        0x00000001042d78f4 chromedriver + 145652
5   chromedriver                        0x00000001042d6f50 chromedriver + 143184
6   chromedriver                        0x00000001042d67f8 chromedriver + 141304
7   chromedriver                        0x00000001042ef69c chromedriver + 243356
8   chromedriver                        0x0000000104341d74 chromedriver + 580980
9   chromedriver                        0x000000010430e318 chromedriver + 369432
10  chromedriver                        0x00000001046271e8 chrome

In [33]:
link = "https://finance.discuss.com.hk/forumdisplay.php?fid=57&filter=type&orderby=new_lastpost&ascdesc=DESC&typeid=1293"

data = {"Date":[], "Topic":[], "Comments":[]}

for p in range(1, 6):
    if p > 0:
        link1 = f"{link}&page={p}"
        page = requests.get(link1)
        
        soup = BeautifulSoup(page.text, 'html.parser')
        all_post = soup.find_all(lambda tag: tag.name == 'tbody' and tag.get('class') == ['forumdisplay_thread'])
        # all_post = soup.find_all("tbody", attrs={"class":"forumdisplay_thread"})

        for h in all_post:
            url = f"https://finance.discuss.com.hk/{h.span.a.get('href')}"
            page = requests.get(url)

            # ads_elems = driver.find_elements(By.CSS_SELECTOR, ".adsbygoogle.adsbygoogle-noablate[data-vignette-loaded=true]")
            # if len(ads_elems) > 0:
            #     ActionChains(driver).move_by_offset(5, 5).click().perform()
            #     time.sleep(5)

            # login_elems = driver.find_elements(By.CSS_SELECTOR, ".maintable.wrap.discuss_regist_wapper login.discuss_regist_section")
            soup = BeautifulSoup(page.text, 'html.parser')
            login_elems = soup.find("div", attrs={"class":"discuss_regist_section"})
            if login_elems != None:
                pass
                # driver.find_elements(By.CSS_SELECTOR, ".discuss_regist_section form input[name='username']")[0].send_keys(ID)
                # driver.find_elements(By.CSS_SELECTOR, ".discuss_regist_section form input[name='password']")[0].send_keys(password)
                # driver.find_elements(By.CSS_SELECTOR, ".discuss_regist_section form button[name='loginsubmit']")[0].click()
            else:
                # soup = BeautifulSoup(page.text, 'html.parser')
                head_comment = soup.find("meta", {"property":"og:description"})["content"]
                head_date = soup.find("div", attrs={"class":"post-date"}).text
                Topic_title = soup.find("span", attrs={"class":"topbar_tid"}).text
                data["Date"].append(head_date)
                data["Topic"].append(Topic_title)
                data["Comments"].append(head_comment)
                all_comments = soup.find_all("div", attrs={"class":"mainbox viewthread"})

                for c in all_comments:
                    if c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span") == None:
                        pass
                    else:
                        if c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span").find("div", attrs={"class":"quote"}) != None:
                            normal_comments = c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span")
                            unwanted = normal_comments.find("div", attrs={"class":"quote"})
                            unwanted.extract()
                            comment_date = c.find("div", attrs={"class":"post-date"}).text
                            Topic_title = soup.find("span", attrs={"class":"topbar_tid"}).text
                            data["Date"].append(comment_date)
                            data["Topic"].append(Topic_title)
                            data["Comments"].append(normal_comments.text)
                        elif c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span").find("div", attrs={"class":"quote"}) == None:
                            normal_comments = c.find("div", attrs={"class":"postmessage-content t_msgfont"}).find("span").text
                            comment_date = c.find("div", attrs={"class":"post-date"}).text
                            Topic_title = soup.find("span", attrs={"class":"topbar_tid"}).text
                            data["Date"].append(comment_date)
                            data["Topic"].append(Topic_title)
                            data["Comments"].append(normal_comments)

                exist_pages = soup.find("div", attrs={"class":"pagination-buttons"})

                if exist_pages == None:
                    pass
                else:
                    no_pages = math.ceil((int(soup.find("div", attrs={"class":"pagination-buttons"}).find("em").text))/15)
                    for i in range(no_pages+1):
                        if i > 1:
                            # ads_elems = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.adsbygoogle.adsbygoogle-noablate[data-vignette-loaded=true]')))
                            # ads_elems = driver.find_elements(By.CSS_SELECTOR, ".adsbygoogle.adsbygoogle-noablate[data-vignette-loaded=true]")
                            # if len(ads_elems) > 0:
                            #     ActionChains(driver).move_by_offset(5, 5).click().perform()
                            #     time.sleep(5)
                            url2 = f'{url}&page={i}'
                            page = requests.get(url2)
                            soup = BeautifulSoup(page.text, 'html.parser')
                            all_comments = soup.find_all("div", attrs={"class":"postmessage-content t_msgfont"})
                            comment_date = soup.find_all("div", attrs={"class":"post-date"})
                            for c in all_comments:
                                if c.find("span") == None:
                                    data["Comments"].append("NA")
                                    Topic_title = soup.find("span", attrs={"class":"topbar_tid"}).text
                                    data["Topic"].append(Topic_title)
                                elif c.find("span").find("div", attrs={"class":"quote"}) != None:
                                    comment = c.find("span")
                                    unwanted = comment.find("div", attrs={"class":"quote"})
                                    unwanted.extract()
                                    Topic_title = soup.find("span", attrs={"class":"topbar_tid"}).text
                                    data["Topic"].append(Topic_title)
                                    data["Comments"].append(comment.text)
                                elif c.find("span").find("div", attrs={"class": "quote"}) == None:
                                    comment = c.find("span").text
                                    Topic_title = soup.find("span", attrs={"class":"topbar_tid"}).text
                                    data["Topic"].append(Topic_title)
                                    data["Comments"].append(comment)
                            for d in comment_date:
                                data["Date"].append(d.text)
                                

In [34]:
data = pd.DataFrame(data)
data1 = data.copy()
data1

,Date,Topic,Comments
0,\n\n ...,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,美股跌幅收窄，道指，標普5連跌，穿20天線。納指倒升。VIX也跌。\r\n【儲局官員】沃勒、...
1,\n\n ...,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,大陸數據確實不太安全
2,\n\n ...,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,真係揾黎搞。阿里已經搖搖欲墜，再搞多幾野，咩信心都散......\n\n真係唔知諗乜。\n\...
3,\n\n ...,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,睇黎阿爺要全方面打阿里
4,\n\n ...,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,阿爺放過阿里啦，大國小氣小家到咁
...,...,...,...
13696,\n\n ...,《紐約郵報》：經濟衰退的恐慌籠罩華爾街，道琼斯指數暴跌,你阿爺同俄佬玩自閉，鬥衰
13697,\n\n ...,《紐約郵報》：經濟衰退的恐慌籠罩華爾街，道琼斯指數暴跌,mola
13698,\n\n ...,NDX put: 10436,美國都未跌完\r\n彈下係最後逃生門 《香港討論區》
13699,\n\n ...,NDX put: 10436,美股只有一個方向


In [35]:
def date_transform(row):
    return row.split("\n")[3].split(" ")[-2]

In [1122]:
# for i in range(len(data1["Comments"])):
#     if "發表" in data1["Comments"][i]:
#         data1["Comments"][i] = ' '.join(data1["Comments"][i].split("\n")[1:])
#     else:
#         data1["Comments"][i] = data1["Comments"][i]

In [36]:
data1["Date"] = data1["Date"].apply(date_transform)
data1["Date"] = pd.to_datetime(data1["Date"])


In [37]:
data1.set_index("Date")

,Topic,Comments
Date,,
2022-07-15,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,美股跌幅收窄，道指，標普5連跌，穿20天線。納指倒升。VIX也跌。\r\n【儲局官員】沃勒、...
2022-07-15,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,大陸數據確實不太安全
2022-07-15,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,真係揾黎搞。阿里已經搖搖欲墜，再搞多幾野，咩信心都散......\n\n真係唔知諗乜。\n\...
2022-07-15,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,睇黎阿爺要全方面打阿里
2022-07-15,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,阿爺放過阿里啦，大國小氣小家到咁
...,...,...
2022-07-06,《紐約郵報》：經濟衰退的恐慌籠罩華爾街，道琼斯指數暴跌,你阿爺同俄佬玩自閉，鬥衰
2022-07-06,《紐約郵報》：經濟衰退的恐慌籠罩華爾街，道琼斯指數暴跌,mola
2022-06-29,NDX put: 10436,美國都未跌完\r\n彈下係最後逃生門 《香港討論區》


In [38]:
data1 = data1[data1["Date"] >= datetime.strptime("2022-07-10", '%Y-%m-%d')].sort_values("Date")

In [22]:
data1 = data1.drop(data1[data1["Date"] == "2022-07-17"].index)

In [40]:
data1.to_csv("Discuss_comment.csv")

In [39]:
data1

,Date,Topic,Comments
11415,2022-07-10,美國經濟降溫！摩根資管︰聯儲局或後悔加息過急,通脹…一直居高不下，反而先係最危險！\n\r\n財長…及聯儲局長，都話會放棄市民，大手加息壓...
9579,2022-07-10,加拿大經濟大簫條,Screenshot_20220710_192818.jpg (728.82 KB)2...
9580,2022-07-10,加拿大經濟大簫條,因為最近好多老尾和老墨過嚟大温隊cocaine，所以生意好旺，連帶啲陀地港女都好搶手，道友多...
9581,2022-07-10,加拿大經濟大簫條,係大温呢喲真係超筍工，好多香港專業人仕，例如社工，財演都爭住做
9582,2022-07-10,加拿大經濟大簫條,中畀向威權統治\r\n那有民主
...,...,...,...
36,2022-07-16,道指反彈658點 恆指會跟嗎？,https://news.tvb.com/tc/instant/ ... utm_mediu...
35,2022-07-16,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,因為仲未踢走軟銀
34,2022-07-16,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,又要等阿爺出實招，再吾減息，放水，就搞吾掂
32,2022-07-16,美期反彈；恒指IG試20400，金管局買入53.22億港元，阿里美股低港6.3%,6月很多利好消息，抄高左，7月大陸就爆鑊
